# Initial Import

In [1]:
from tqdm import tqdm
import os
import numpy as np
import torch
from torch import optim
from torch.utils.data import Dataset, DataLoader, TensorDataset, random_split
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd

# from main import backgammon

/Applications/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


## Import Dataset

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

# Load data
def load_dataset(path):
    data = np.loadtxt(path, delimiter=",")
    X = torch.tensor(data[:, :-1], dtype=torch.float32)
    y = torch.tensor(data[:, -1], dtype=torch.float32).unsqueeze(1)
    return TensorDataset(X, y)

train_data = load_dataset(os.path.join("..","Data","Deep","Cube","Accept","train.txt"))
test_data = load_dataset(os.path.join("..","Data","Deep","Cube","Accept","test.txt"))

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64)

# Model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(291, 12)
        self.fc2 = nn.Linear(12, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        return x

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Net().to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
for epoch in range(1, 101):
    model.train()
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if epoch % 10 == 0:
        model.eval()
        correct = total = 0
        with torch.no_grad():
            for X_test, y_test in test_loader:
                X_test, y_test = X_test.to(device), y_test.to(device)
                outputs = model(X_test)
                predictions = (outputs > 0.5).float()
                correct += (predictions == y_test).sum().item()
                total += y_test.size(0)
        accuracy = correct / total
        print(f"Epoch {epoch}, Test Accuracy: {accuracy:.4f}")
torch.save(model.state_dict(), "cube_accept_model.pth")

torch.Size([64, 289])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (64x289 and 291x12)